# Compito di classificazione del testo

In questo modulo, inizieremo con un semplice compito di classificazione del testo basato sul dataset **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: classificheremo i titoli delle notizie in una delle 4 categorie: Mondo, Sport, Economia e Scienza/Tecnologia.

## Il Dataset

Per caricare il dataset, utilizzeremo l'API **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Possiamo ora accedere alle parti di addestramento e di test del dataset utilizzando rispettivamente `dataset['train']` e `dataset['test']`:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Stampiamo i primi 10 nuovi titoli dal nostro dataset:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Vettorizzazione del testo

Ora dobbiamo convertire il testo in **numeri** che possano essere rappresentati come tensori. Se vogliamo una rappresentazione a livello di parola, dobbiamo fare due cose:

* Usare un **tokenizzatore** per suddividere il testo in **token**.
* Costruire un **vocabolario** di quei token.

### Limitare la dimensione del vocabolario

Nell'esempio del dataset AG News, la dimensione del vocabolario è piuttosto grande, con più di 100.000 parole. In generale, non abbiamo bisogno di parole che compaiono raramente nel testo — solo poche frasi le conterranno, e il modello non imparerà da esse. Pertanto, ha senso limitare la dimensione del vocabolario a un numero più piccolo passando un argomento al costruttore del vettorizzatore:

Entrambi questi passaggi possono essere gestiti utilizzando il livello **TextVectorization**. Creiamo l'oggetto vettorizzatore e poi chiamiamo il metodo `adapt` per analizzare tutto il testo e costruire un vocabolario:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Nota** che stiamo utilizzando solo un sottoinsieme dell'intero dataset per costruire un vocabolario. Lo facciamo per velocizzare il tempo di esecuzione e non farti aspettare. Tuttavia, corriamo il rischio che alcune parole dell'intero dataset non vengano incluse nel vocabolario e vengano ignorate durante l'addestramento. Pertanto, utilizzare l'intera dimensione del vocabolario e scorrere tutto il dataset durante `adapt` dovrebbe aumentare l'accuratezza finale, ma non in modo significativo.

Ora possiamo accedere al vocabolario effettivo:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Utilizzando il vettorizzatore, possiamo facilmente codificare qualsiasi testo in un insieme di numeri:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Rappresentazione testuale Bag-of-words

Poiché le parole rappresentano significati, a volte possiamo capire il senso di un testo semplicemente osservando le singole parole, indipendentemente dal loro ordine nella frase. Ad esempio, quando si classificano notizie, parole come *meteo* e *neve* probabilmente indicano una *previsione del tempo*, mentre parole come *azioni* e *dollaro* sarebbero associate a *notizie finanziarie*.

La rappresentazione vettoriale **Bag-of-words** (BoW) è la più semplice da comprendere tra le rappresentazioni vettoriali tradizionali. Ogni parola è collegata a un indice vettoriale, e un elemento del vettore contiene il numero di occorrenze di ciascuna parola in un determinato documento.

![Immagine che mostra come una rappresentazione vettoriale bag-of-words è rappresentata in memoria.](../../../../../translated_images/it/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Nota**: Puoi anche pensare al BoW come alla somma di tutti i vettori one-hot-encoded per le singole parole nel testo.

Di seguito è riportato un esempio di come generare una rappresentazione bag-of-words utilizzando la libreria python Scikit Learn:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Possiamo anche utilizzare il vettorizzatore Keras che abbiamo definito sopra, convertendo ogni numero di parola in una codifica one-hot e sommando tutti quei vettori.


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Nota**: Potresti essere sorpreso dal fatto che il risultato differisca dall'esempio precedente. La ragione è che nell'esempio di Keras la lunghezza del vettore corrisponde alla dimensione del vocabolario, che è stato costruito sull'intero dataset AG News, mentre nell'esempio di Scikit Learn abbiamo costruito il vocabolario direttamente dal testo di esempio.


## Addestrare il classificatore BoW

Ora che abbiamo imparato a costruire la rappresentazione bag-of-words del nostro testo, alleniamo un classificatore che la utilizza. Per prima cosa, dobbiamo convertire il nostro dataset in una rappresentazione bag-of-words. Questo può essere fatto utilizzando la funzione `map` nel seguente modo:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Ora definiamo una semplice rete neurale classificatrice che contiene un unico strato lineare. La dimensione dell'input è `vocab_size`, e la dimensione dell'output corrisponde al numero di classi (4). Poiché stiamo risolvendo un compito di classificazione, la funzione di attivazione finale è **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Dal momento che abbiamo 4 classi, un'accuratezza superiore all'80% è un buon risultato.

## Addestrare un classificatore come una rete unica

Poiché il vettorizzatore è anche un livello di Keras, possiamo definire una rete che lo include e addestrarla end-to-end. In questo modo, non è necessario vettorizzare il dataset utilizzando `map`, possiamo semplicemente passare il dataset originale come input della rete.

> **Nota**: Dovremmo comunque applicare mappe al nostro dataset per convertire i campi dai dizionari (come `title`, `description` e `label`) in tuple. Tuttavia, quando carichiamo i dati dal disco, possiamo costruire un dataset con la struttura richiesta fin dall'inizio.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigrammi, trigrammi e n-grammi

Una limitazione dell'approccio bag-of-words è che alcune parole fanno parte di espressioni composte da più termini. Ad esempio, la parola 'hot dog' ha un significato completamente diverso rispetto alle parole 'hot' e 'dog' prese singolarmente in altri contesti. Se rappresentiamo sempre le parole 'hot' e 'dog' utilizzando gli stessi vettori, potremmo confondere il nostro modello.

Per affrontare questo problema, si utilizzano spesso le **rappresentazioni n-gram** nei metodi di classificazione dei documenti, dove la frequenza di ogni parola, coppia di parole o terzina di parole rappresenta una caratteristica utile per addestrare i classificatori. Nelle rappresentazioni bigrammi, ad esempio, aggiungiamo al vocabolario tutte le coppie di parole, oltre alle parole originali.

Di seguito è riportato un esempio di come generare una rappresentazione bag-of-words con bigrammi utilizzando Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Lo svantaggio principale dell'approccio n-gram è che la dimensione del vocabolario inizia a crescere estremamente rapidamente. In pratica, è necessario combinare la rappresentazione n-gram con una tecnica di riduzione della dimensionalità, come le *embedding*, di cui parleremo nella prossima unità.

Per utilizzare una rappresentazione n-gram nel nostro dataset **AG News**, dobbiamo passare il parametro `ngrams` al costruttore di `TextVectorization`. La lunghezza di un vocabolario di bigrammi è **significativamente più grande**, nel nostro caso supera 1,3 milioni di token! Pertanto, ha senso limitare anche i token dei bigrammi a un numero ragionevole.

Potremmo utilizzare lo stesso codice di cui sopra per addestrare il classificatore, tuttavia, sarebbe molto inefficiente in termini di memoria. Nella prossima unità, addestreremo il classificatore di bigrammi utilizzando le embedding. Nel frattempo, puoi sperimentare l'addestramento del classificatore di bigrammi in questo notebook e vedere se riesci a ottenere una precisione maggiore.


## Calcolo automatico dei vettori BoW

Nell'esempio sopra abbiamo calcolato i vettori BoW manualmente sommando le codifiche one-hot delle singole parole. Tuttavia, l'ultima versione di TensorFlow ci consente di calcolare automaticamente i vettori BoW passando il parametro `output_mode='count` al costruttore del vettorizzatore. Questo rende la definizione e l'addestramento del nostro modello significativamente più semplice:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Frequenza termine - frequenza inversa del documento (TF-IDF)

Nella rappresentazione BoW, le occorrenze delle parole vengono pesate utilizzando la stessa tecnica indipendentemente dalla parola stessa. Tuttavia, è evidente che parole frequenti come *a* e *in* sono molto meno importanti per la classificazione rispetto ai termini specializzati. Nella maggior parte dei compiti di NLP alcune parole sono più rilevanti di altre.

**TF-IDF** sta per **frequenza termine - frequenza inversa del documento**. È una variazione del bag-of-words, dove invece di un valore binario 0/1 che indica la presenza di una parola in un documento, viene utilizzato un valore in virgola mobile, che è correlato alla frequenza di occorrenza della parola nel corpus.

Più formalmente, il peso $w_{ij}$ di una parola $i$ nel documento $j$ è definito come:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
dove
* $tf_{ij}$ è il numero di occorrenze di $i$ in $j$, ovvero il valore BoW che abbiamo visto prima
* $N$ è il numero di documenti nella collezione
* $df_i$ è il numero di documenti che contengono la parola $i$ nell'intera collezione

Il valore TF-IDF $w_{ij}$ aumenta proporzionalmente al numero di volte in cui una parola appare in un documento ed è compensato dal numero di documenti nel corpus che contengono la parola, il che aiuta a correggere il fatto che alcune parole appaiono più frequentemente di altre. Ad esempio, se la parola appare in *ogni* documento della collezione, $df_i=N$, e $w_{ij}=0$, e quei termini verrebbero completamente ignorati.

Puoi facilmente creare una vettorizzazione TF-IDF del testo utilizzando Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In Keras, il livello `TextVectorization` può calcolare automaticamente le frequenze TF-IDF passando il parametro `output_mode='tf-idf'`. Ripetiamo il codice che abbiamo utilizzato sopra per vedere se l'uso di TF-IDF aumenta l'accuratezza:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Conclusione

Anche se le rappresentazioni TF-IDF assegnano pesi di frequenza a diverse parole, non sono in grado di rappresentare il significato o l'ordine. Come disse il famoso linguista J. R. Firth nel 1935: "Il significato completo di una parola è sempre contestuale, e nessuno studio del significato al di fuori del contesto può essere preso sul serio." Più avanti nel corso impareremo come catturare le informazioni contestuali dai testi utilizzando il language modeling.



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
